[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Image Processing with Python

## Image Contrast and Point Operations

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 05/10/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0003Contrast.ipynb)

In [1]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [2]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [3]:
# Constants



In [4]:
# Fixel Algorithms Packages

from AuxFun import *


In [5]:
# General Auxiliary Functions



## Image Contrast & Point Operations

Image point operations are basic tools for image manipulation.  
This notebook exercises applying some 1D transformations on data in order to achieve a _Global Contrast Enhancement_.

* <font color='brown'>(**#**)</font> The concept is sometimes attributed to [Gustav J. Braun, “Image Lightness Rescaling Using Sigmoidal Contrast Enhancement Functions” (1998)](https://doi.org/10.1117/12.334548).
* <font color='brown'>(**#**)</font> The same concept can be used for Masked Color Saturation Enhancement (_Vibrance_) by applying on the `a` / `u` and `b` / `v` channel of `LAB` / `YUV`.

### The Image

For this notebook an image by the _underwater photographer_ [Paolo Fossati](http://www.paolo-fossati.com) will be used.

![](https://i.imgur.com/j4QZWiL.png)

In [6]:
# Parameters

# imgUrl = 'https://i.imgur.com/j4QZWiL.png' #<! By Paolo Fossati
imgUrl = 'https://i.postimg.cc/c4RCtxfd/j4QZWiL.jpg' #<! By Paolo Fossati

In [7]:
# Load the Image

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float(mI) #<! Convert to Float


### Image Contrast

This section applies _global contrast_ enhancement using S function like.  
When enhancing images, masking is commonly used.  
The masking allows having different effect level on different luminosity levels.

1. Implement S function  
   One of the choices: Logistic (Sigmoid), Error Function, Hyperbolic Tangent, Smooth Step, etc...
2. Implement Masking function.
3. Apply on the image.
4. Display results.


#### Luminosity Masks

![](https://fixelalgorithms.co/news/images/LuminosityMask001/LuminosityMaskShowCaseAnimated.png)

In [8]:
#===========================Fill This===========================#
# 1. Implement S Function.
# !! Numba can be used.

def ApplySFunction( tA: np.ndarray, α: float, tO: np.ndarray ):
    """
    Applies an S function on the input array.
    Input:
      - mI           : A NumPy array at any shape with floating point values in the range [0, 1].
      - α            : A float for the intensity of the operation (Slope).
      - tO           : A NumPy array with the same shape and dtype as the input.
    Remarks:
      - The output range should be [0, 1].
      - The function should support the case `tA` and `tO` are the same (Inplace).
    """
    
    tO[:] = np.reciprocal(1 + np.exp(-α * (tA - 0.5)))

#===============================================================#

In [9]:
#===========================Fill This===========================#
# 1. Implement a Luminosity Mask function.
# 2. The function should support: Highlights, Shadows and Midtones.
# 3. The zones are defined on [0, 1] as following:
#    - Shadows: f(x) = 1 - x.
#    - Highlights: f(x) = x.
#    - Midtones: *Scaled* multiplication of Highlights and Midtones.
#    Each of the ranges spans the whole range of [0, 1]. 
# !! Numba can be used.

def GenerateLuminosityMask( mL: np.ndarray, maskType: int, mM: np.ndarray ):
    """
    Generate a luminosity mask for the input luminosity image.
    Input:
      - mL           : A NumPy 2D array in the range [0, 1].
      - maskType     : An integer: 0 -> Shadows, 1 -> Midtones, 2 -> Highlights.
      - mM           : A NumPy 2D array with the same shape and dtype as the input.
    Remarks:
      - The output range should be [0, 1].
      - The function should support the case `mL` and `mM` are the same (Inplace).
    """
    
    if maskType == 0:
        mM[:] = 1 - mL
    elif maskType == 1:
        mM[:] = 4 * mL * (1 - mL)
    elif maskType == 2:
        mM[:] = mL

#===============================================================#

In [10]:
def VisImgContrast( mI: np.ndarray, α: float, maskTypeStr: str, mO: np.ndarray ):
    
    # Generate the Mask
    mM = np.ones(shape = mI.shape[:2]) #<! Default (And allocation)
    maskType = 0
    if maskTypeStr == 'Shadows':
        maskType = 0
    elif maskTypeStr == 'Midtones':
        maskType = 1
    elif maskTypeStr == 'Highlights':
        maskType = 2
    if maskTypeStr != 'All':
        GenerateLuminosityMask(ski.color.rgb2gray(mI), maskType, mM)
    
    # Apply contrast
    ApplySFunction(mI, α, mO)
    
    # Apply Mask
    mO[:] = (mI * (1 - mM[:, :, None])) + (mO * mM[..., None])
    
    hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (16, 5))
    
    vHa[0].imshow(mI)
    vHa[0].set(xticks = [], yticks = [])
    vHa[1].imshow(mM, cmap = 'gray', vmin = 0, vmax = 1)
    vHa[1].set(xticks = [], yticks = [])
    vHa[2].imshow(mO)
    vHa[2].set(xticks = [], yticks = [])

* <font color='red'>(**?**)</font> The above implementation applies the contrast to each RGB channel. Usually, contrast is applied to Luminosity channel only.

In [ ]:
mO = np.zeros_like(mI)

hVisImgContrast = lambda α, maskTypeStr: VisImgContrast(mI, α, maskTypeStr, mO)

αSlider = FloatSlider(min = 0.1, max = 20, step = 0.01, value = 10, layout = Layout(width = '30%'))
maskTypeStrDropdown = Dropdown(options = ['All', 'Shadows', 'Midtones', 'Highlights'], value = 'All', description = 'Luminosity Mask Type:')
interact(hVisImgContrast, α = αSlider, maskTypeStr = maskTypeStrDropdown)

plt.show()

* <font color='red'>(**?**)</font> In the reference implementation, what's the issue for `α < 5`? You may want to plot the contrast function.
* <font color='green'>(**@**)</font> Optimize the contrast function to mitigate the issue.

To understand the _luminosity mask_ one should refer to the histogram.

In [ ]:
mL = ski.color.rgb2gray(mI) #<! Pseudo Luminosity channel
vH, vB = ski.exposure.histogram(mL, normalize = True)

vM = 4 * vB * (1 - vB) #<! Midtones Mask of the Histogram Bins
lC = [plt.get_cmap('viridis')(valM) for valM in vM]

hF, hA = plt.subplots(figsize = (12, 6))
hA.bar(vB, vH, width = 1 / 256, align = 'center', color = lC)
hA.set(yticks = [], xlim = (0, 1), title = 'Luminosity Channel Histogram');

### The `Y` Channel Update Trick

 - Many algorithms are applied only on a _luminosity_ like channel.
 - Hence, they are usually applied by:
   1. Convert to another color space.
   2. Apply effect.
   3. Convert back to `RGB`.
 - In case the luminosity channel is a result of a linear transformation (`YUV`, `YCbCr`, `YCgCr`, etc) this can be optimized.

Using the `YUV` example:

$$\begin{bmatrix} y \\ u \\ v \end{bmatrix} = \boldsymbol{C} \begin{bmatrix} r \\ g \\ b \end{bmatrix} \implies \begin{bmatrix} r \\ g \\ b \end{bmatrix} = \boldsymbol{D} \begin{bmatrix} y \\ u \\ v \end{bmatrix}, \; \boldsymbol{D} = \boldsymbol{C}^{-1}$$

Pay attention that for all cases, for the matrix $\boldsymbol{C}$ the following holds:

 - The sum of the first row $\boldsymbol{c}^{1}$ is one: $\boldsymbol{1}^{T} \boldsymbol{c}^{1} = 1$.
 - The sum of any other row is zero: $i \in \left\{ 2, 3 \right\} \implies \boldsymbol{1}^{T} \boldsymbol{c}^{i} = 0$.

Since $\boldsymbol{C} \boldsymbol{D} = \boldsymbol{I}$ one can conclude that the first column of $\boldsymbol{D}$, given as $\boldsymbol{d}_{1}$, must be $\boldsymbol{1}$ since:

 - It must be constant as the inner product of a zero sum with it is $0$.
 - Its convex sum equals to $1$.

Now, per pixel, by defining $\hat{y}$ the luminosity value after the operation, one can look at the delta for the RGB values:

$$ \boldsymbol{\Delta} = \begin{bmatrix} \hat{r} \\ \hat{g} \\ \hat{b} \end{bmatrix} - \begin{bmatrix} r \\ g \\ b \end{bmatrix} = \boldsymbol{D} \begin{bmatrix} \hat{y} \\ u \\ v \end{bmatrix} - \boldsymbol{D} \begin{bmatrix} y \\ u \\ v \end{bmatrix} = \boldsymbol{D} \begin{bmatrix} \hat{y} - y \\ 0 \\ 0 \end{bmatrix} = \left( \hat{y} - y \right) \boldsymbol{d}_{i} = \left( \hat{y} - y \right) \boldsymbol{1} $$

Instead of calculating all transformations, one can only add $\boldsymbol{\Delta}$ to the RGB image.

In [ ]:
# Verify the Y Channel Updated Trick
vY = np.array([0.299, 0.587, 0.114]) #<! Match SciKit Image `rgb2yuv()` (`rgb2gray()` Doesn't match)
α = 20

#===========================Fill This===========================#
# 1. Generate `mY` using `vY` as a grayscale image.
# 2. Generate  `mYUV` using SciKit Image's `rgb2yuv()`.
# 3. Apply the contrast function using `ApplySFunction()`:
#  - On `mY`.
#  - On the `Y` channel of `mYUV`.
# 4. Calculate the updated RGB image:
#   - Using the `Y` channel trick.
#   - Using `YUV` to `RGB` conversion `yuv2rgb()`.
# 5. Verify the results match.

# Convert
mY   = np.dot(mI, vY) #<! Convert to Y using the weights
mYUV = ski.color.rgb2yuv(mI) #<! Convert to YUV

# Apply transformation
mYHat = np.zeros_like(mY)
ApplySFunction(mY, α, mYHat)
ApplySFunction(mYUV[..., 0], α, mYUV[..., 0])

# Calculate the RGB images
mOTrick = mI + (mYHat - mY)[:, :, None] #<! Trick
mODirect = ski.color.yuv2rgb(mYUV) #<! Classic

#===============================================================#

print(f'The Y channel trick worked: {np.allclose(mOTrick, mODirect)}')

### ScKit Image Exposure Module

The `exposure` module has few other functions which deals with a point wise manipulation of the pixel value.  
This section demonstrates few of them. 

In [ ]:
# Check if the image is a low contrast image

isLowContrast = ski.exposure.is_low_contrast(mI)
print(f'Is the image a low contrast image: {isLowContrast}') #<! By a threshold on the covered range by the image

In [ ]:
# Apply Image Histogram Equalization
# This section applies it on "Luminosity Channel".

#===========================Fill This===========================#
# 1. Convert the image to YUV.
# 2. Apply histogram equalization on the Y channel.
# 3. Convert back to RGB
# !! Look at the `color` module of SciKit Learn.
# !! Due to numerical issues, it is better to clip to a valid range after some operations.
# !! You may rewrite things to use the Y channel trick.

mYUV = ski.color.rgb2yuv(mI)
mYUV[:, :, 0] = np.clip(ski.exposure.equalize_hist(mYUV[..., 0]), a_min = 0.0, a_max = 1.0)
mO = np.clip(ski.color.yuv2rgb(mYUV), a_min = 0.0, a_max = 1.0)

#===============================================================#

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 6))
vHa[0].imshow(mI)
vHa[0].set(xticks = [], yticks = [], title = 'Input Image')
vHa[1].imshow(mO)
vHa[1].set(xticks = [], yticks = [], title = 'Histogram Equalization');

* <font color='brown'>(**#**)</font> You may notice some reduction in saturation after increasing contrast on the luminosity channels.

The `equalize_adapthist` is a local version of the histogram equalization.  
It applies the effect on the luminosity automatically.

In [ ]:
# Contrast Limited Adaptive Histogram Equalization
mO = ski.exposure.equalize_adapthist(mI, clip_limit = 0.005)

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 6))
vHa[0].imshow(mI)
vHa[0].set(xticks = [], yticks = [], title = 'Input Image')
vHa[1].imshow(mO)
vHa[1].set(xticks = [], yticks = [], title = 'CLAHE');

* <font color='brown'>(**#**)</font> SciKit Image's `equalize_adapthist` is actually contrast limited adaptive histogram equalization (_CLAHE_).
* <font color='brown'>(**#**)</font> Histogram equalization and its derivatives usually used in the context of image analysis, less in image enhancement for viewing.
* <font color='blue'>(**!**)</font> Raise the values of `clip_limit` to see the exaggerated local effect.